In [25]:
import yfinance as yf
import talib
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import datetime

In [26]:
# 函数：下载股票数据并计算技术指标
def prepare_stock_data(stock_symbol, start_date, end_date):
    df = yf.download(stock_symbol, start=start_date, end=end_date)
    for ma in [15, 30, 45, 60, 90, 120]:
        df[f'MA_{ma}'] = talib.SMA(df['Close'], timeperiod=ma)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df.dropna(inplace=True)
    return df

In [27]:
# 函数：训练 SVM 模型
def train_svm_model(data):
    X = data[['MA_15', 'MA_30', 'MA_45', 'MA_60', 'MA_90', 'MA_120', 'RSI']]
    y = data['Close']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    svm_model = SVR(kernel='rbf')
    svm_model.fit(X_train_scaled, y_train)
    return svm_model, scaler

In [28]:
# 选择多个股票用于训练
stock_symbols = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'META', 'AMD', 'DLR', 
                 'VOO', 'SPY', 'MS', 'JPM', 'NVO', 'UNH', 'AMZN', 'WMT', 'BA', 
                 'BRK', 'TLT', 'HYG', 'MCO', 'ASML', 'GE', 'MC.PA']  # 示例股票代码
start_date = '2000-01-01'
# end_date = '2023-11-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# 汇总多个股票数据
combined_data = pd.DataFrame()
for symbol in stock_symbols:
    stock_data = prepare_stock_data(symbol, start_date, end_date)
    combined_data = combined_data._append(stock_data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [29]:
# 训练模型
svm_model, scaler = train_svm_model(combined_data)

In [30]:
# 使用模型对特定股票进行预测
target_stock = 'GOOG'  # 需要预测的股票
target_data = prepare_stock_data(target_stock, start_date, end_date)
target_features = target_data[['MA_15', 'MA_30', 'MA_45', 'MA_60', 'MA_90', 'MA_120', 'RSI']]
target_features_scaled = scaler.transform(target_features)

# 进行预测
target_predictions = svm_model.predict(target_features_scaled)

# 显示预测结果的最后几项
predicted_prices = pd.DataFrame(target_predictions, index=target_data.index, columns=['Predicted Close'])
predicted_prices.tail()

[*********************100%%**********************]  1 of 1 completed


,Predicted Close
Date,
2023-11-16,138.597962
2023-11-17,133.290738
2023-11-20,135.301451
2023-11-21,137.037091
2023-11-22,140.856617
